# Hệ khuyến nghị. IUH 2025.
### Ngày 02/10/2025. Lab 6.
Mục tiêu: knowledge-base recommendation system.

### Bài 1. 

Tìm hiểu một ví dụ về các bước thực hiện khuyến nghị dựa trên tri thức. Bài lab tham khảo từ:

https://michael-fuchs-python.netlify.app/2020/10/01/recommendation-systems-knowledged-based-recommender

In [1]:
from ast import literal_eval
import pandas as pd

file_url = 'https://github.com/MFuchs1989/Datasets-and-Miscellaneous/raw/main/datasets/Recommendation%20Systems/Knowledged-based%20Recommender/movies_metadata.csv'

df = pd.read_csv(file_url)

# Select just relevant features
relevant_features = ['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']
df = df[relevant_features]

# Print the dataframe
df.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17884\2335380705.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_url)


,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [2]:
import numpy as np
# Pre process data

#Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Extract year from release_date-column and store the values into a new year-column
df['year'] = pd.DatetimeIndex(df['release_date']).year

#Convert_int to the year feature
df['year'] = df['year'].fillna(0).astype(int)

#Drop the release_date column
df = df.drop('release_date', axis=1)


#Display the dataframe
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [3]:
#Convert all NaN into stringified empty lists
df['genres'] = df['genres'].fillna('[]')

#Apply literal_eval to convert stringified empty lists to the list object
df['genres'] = df['genres'].apply(literal_eval)

#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


In [4]:
#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

#Name the new feature as 'genre'
s.name = 'genre'

#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(s)

#Print the head of the new gen_df
gen_df.head()

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy


In [5]:
def build_chart(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C), axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [ ]:
personal_recommendations = build_chart(gen_df).head(8)
personal_recommendations.head()

Input preferred genre


### Bài 2.

Building a Knowledge-Based Recommender with the Book-Crossing Dataset: This dataset includes information about books, such as titles, authors, publication years, publishers, and user ratings, making it ideal for creating a recommendation system based on user preferences.

Link: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset

Important Columns for the Recommender
- Book-Title: Main identifier for each book.
- Book-Author: Provides a filter for books by a specific author.
- Year-Of-Publication: Allows filtering by publication year, useful for users looking for books from a particular period.
- Publisher: Enables filtering based on the publisher.
- Book-Rating (from Ratings.csv): Helps refine recommendations by including only higher-rated books, based on user preferences.

### Steps for Building the Recommender.

This addendum adapts the original recommender code to work with the Book-Crossing Dataset, using the correct file names and column names. Here’s how each step is applied:

#### Step 1: Data Preparation

Download and load the dataset using kagglehub. We’ll merge the book information from Books.csv with ratings data from Ratings.csv to incorporate Book-Rating for each book.

In [ ]:
import pandas as pd
import kagglehub

# Download the Book-Crossing dataset from Kaggle
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

# Display the path for reference
print("Path to dataset files:", path)

# Load book information and ratings data
books = pd.read_csv(path + "/Books.csv", encoding='ISO-8859-1', sep=';', on_bad_lines='skip')
ratings = pd.read_csv(path + "/Ratings.csv", encoding='ISO-8859-1', sep=';', on_bad_lines='skip')

# Display the columns in each file to verify
print("Books columns:", books.columns)
print("Ratings columns:", ratings.columns)

# Attempt to merge if both contain the 'ISBN' column
if 'ISBN' in books.columns and 'ISBN' in ratings.columns:
    # Merge books and ratings data on ISBN
    df = books.merge(ratings, on='ISBN', how='left')

    # Check the columns in the merged DataFrame
    print("Merged DataFrame columns:", df.columns)

    # Adjust feature selection to match the exact column names
    relevant_features = ['Book-Title', 'Book-Author', 'Publisher', 'Year-Of-Publication', 'Book-Rating']
    df = df[[col for col in relevant_features if col in df.columns]]
    df.dropna(inplace=True)  # Drop rows with missing values
    print("Dataset Shape:", df.shape)
    print(df.head())
else:
    print("ISBN column is missing in one of the files, cannot merge.")

**Explanation:** We load the Books.csv and Ratings.csv files, merge them on ISBN to combine book data with ratings, and retain only the necessary columns for recommendations.

#### Step 2: Define the Recommendation Function

Create a function to filter books based on user-defined preferences. This function will apply filtering rules for attributes like author, publisher, publication year, and rating.

In [ ]:
def get_recommendations(preferences, data):
    """
    Filters books based on user preferences.

    Parameters:
    - preferences: A dictionary containing user-defined feature preferences
    - data: The DataFrame containing book data

    Returns:
    - DataFrame: Filtered items matching the preferences
    """
    filtered_data = data.copy()
    for feature, value in preferences.items():
        if value:
            if feature == 'Book-Rating':  # Filter by minimum rating
                filtered_data = filtered_data[filtered_data[feature] >= value]
            elif feature == 'Year-Of-Publication':  # Filter by exact publication year
                filtered_data = filtered_data[filtered_data[feature] == value]
            else:
                # Filter by partial matches for text-based columns
                filtered_data = filtered_data[filtered_data[feature].str.contains(value, case=False, na=False)]
    return filtered_data

**Explanation:**

- This function iterates over each user preference and applies filters accordingly.
- Book-Rating is treated as a minimum threshold, and Year-Of-Publication is used for exact matches.
- Text-based columns like Book-Author and Publisher allow partial matches for flexibility.


#### Step 3: Define User Preferences

Define a sample set of preferences to customize recommendations. Adjust these values to reflect user-specific inputs for attributes like author, publisher, publication year, and minimum rating.

In [ ]:
user_preferences = {
    'Book-Author': 'J.K. Rowling',     # User's preferred author
    'Publisher': 'Scholastic Inc.',    # User's preferred publisher
    'Year-Of-Publication': 2000,       # Specific publication year
    'Book-Rating': 4                   # Minimum rating threshold
}

**Explanation:** This dictionary defines user preferences for each feature. Users can specify values for any attribute or leave attributes blank to avoid filtering on them.

#### Step 4: Get Recommendations
Finally, apply the get_recommendations function with the dataset and preferences to obtain the filtered results.

In [ ]:
# Get recommendations based on preferences
recommendations = get_recommendations(user_preferences, df)

# Display recommendations
print("Recommended Books:")
print(recommendations.head())

**Explanation:** This code filters the books based on the provided preferences and displays the top results that align with the user’s choices.

**Final Thoughts**
This addendum demonstrates how to adapt the knowledge-based recommender for the Book-Crossing Dataset, using the correct file names and paths. By following these steps, you can implement similar logic for other datasets by adjusting the feature selection and user preferences to match each dataset’s attributes. Try applying this process to other datasets and explore how each feature affects the recommendations.

#### Criteria for Choosing a Suitable Dataset

To effectively use this recommendation approach, you’ll need a dataset that meets the following criteria:

- Descriptive Item Attributes: The dataset should contain attributes that provide meaningful descriptions of each item, such as category, genre, or price.
- Diverse Feature Types: Look for a mix of categorical (e.g., genre, location) and numerical (e.g., rating, price) features. - - Categorical features make it easier to define user preferences, while numerical features help add depth to recommendations.
- Clear Preference Mapping: The dataset’s attributes should be relatable for filtering preferences. For instance, users might want to filter books by genre, hotels by location, or products by price.
- Sufficient Data Quantity: Aim for datasets with a substantial number of rows (at least 500 items) and varied values in each attribute to ensure diversity in recommendations.

#### Bài 3. Recommended Datasets on Kaggle

Below are three datasets from Kaggle that are compatible with this approach. Each dataset includes attributes that align well with user-defined preferences for building a recommender. Try creating a recommender using each of these datasets to practice and strengthen your skills.

> SV chọn 1 trong 3 dataset bên dưới và thực hiện 1 mô hình hệ khuyến nghị sử dụng knowledge-base theo các gợi ý tương ứng.

#### 1. Book-Crossing Dataset

- Link: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset

- Description: This dataset contains book ratings, authors, publishers, and publication years, making it ideal for building a book recommender based on user preferences.
  
- Using Important Columns in Recommender Steps: bookTitle, bookAuthor, publisher, yearOfPublication, bookRating.

- Steps for Building the Recommender:

*Data Preparation*: Load the dataset, and inspect bookTitle, bookAuthor, publisher, yearOfPublication, and bookRating.

*Feature Selection*: Select bookTitle as the main identifier, with other columns for filtering.

*Filtering Logic*: Implement filters for bookAuthor, publisher, yearOfPublication, and bookRating based on user input.

*User Preferences*: Allow users to specify preferences for each of the selected columns.

*Recommendation Output*: Display filtered books that match user preferences.

In [7]:
import pandas as pd

books = pd.read_csv("LAB06\Books.csv", sep=';', on_bad_lines='skip', encoding="latin-1")
ratings = pd.read_csv("LAB06\Ratings.csv", sep=';', on_bad_lines='skip', encoding="latin-1")

# Xem qua dữ liệu
print(books.head(10))
print(ratings.head(10))


  ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0  0195153448,Classical Mythology,Mark P. O. Morf...                                           
1  0002005018,Clara Callan,Richard Bruce Wright,2...                                           
2  0060973129,Decision in Normandy,Carlo D'Este,1...                                           
3  0374157065,Flu: The Story of the Great Influen...                                           
4  0399135782,The Kitchen God's Wife,Amy Tan,1991...                                           
5  0425176428,What If?: The World's Foremost Mili...                                           
6  0671870432,PLEADING GUILTY,Scott Turow,1993,Au...                                           
7  0679425608,Under the Black Flag: The Romance a...                                           
8  074322678X,Where You'll Find Me: And Other Sto...                                           
9  0771074670,Nights Below Station Stree

In [24]:
import pandas as pd

# Đọc file Books.csv
books = pd.read_csv("LAB06/Books.csv", sep=",", encoding="latin-1", low_memory=False)

# Đọc file Ratings.csv
ratings = pd.read_csv("LAB06/Ratings.csv", sep=",", encoding="latin-1", low_memory=False)

# In thử tên cột để chắc chắn
print("Books columns:", books.columns.tolist())
print("Ratings columns:", ratings.columns.tolist())

# Chọn cột cần thiết
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]

# Gộp 2 bảng
data = pd.merge(ratings, books, on='ISBN')

print(data.head())


# 4. Hàm gợi ý sách
def recommend_books(author=None, publisher=None, year=None, min_rating=None, top_n=10):
    df = data.copy()
    
    if author:
        df = df[df['Book-Author'].str.contains(author, case=False, na=False)]
    if publisher:
        df = df[df['Publisher'].str.contains(publisher, case=False, na=False)]
    if year:
        df = df[df['Year-Of-Publication'] == year]
    if min_rating:
        df = df[df['Book-Rating'] >= min_rating]
    
    # Lấy sách phổ biến nhất theo trung bình rating
    result = df.groupby(['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher'])['Book-Rating'] \
               .mean().reset_index().sort_values(by='Book-Rating', ascending=False)
    
    return result.head(top_n)

# ========================
# 5. Ví dụ test
# ========================

print("📚 Sách của J.K. Rowling có rating >= 5:")
print(recommend_books(author="J.K. Rowling", min_rating=5, top_n=5))

print("\n📚 Sách của nhà xuất bản Penguin năm 2000:")
print(recommend_books(publisher="Penguin", year=2000, top_n=5))



Books columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
Ratings columns: ['User-ID', 'ISBN', 'Book-Rating']
   User-ID        ISBN  Book-Rating  \
0   276725  034545104X            0   
1   276726  0155061224            5   
2   276727  0446520802            0   
3   276729  052165615X            3   
4   276729  0521795028            6   

                                          Book-Title      Book-Author  \
0                               Flesh Tones: A Novel       M. J. Rose   
1                                   Rites of Passage       Judith Rae   
2                                       The Notebook  Nicholas Sparks   
3                                     Help!: Level 1    Philip Prowse   
4  The Amsterdam Connection : Level 4 (Cambridge ...      Sue Leather   

  Year-Of-Publication                   Publisher  
0                2002            Ballantine Books  
1                2001               

In [26]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
import kagglehub

# ==============================
# 1. Tải dataset từ Kaggle
# ==============================
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")
print("Path to dataset files:", path)

# ==============================
# 2. Đọc dữ liệu
# ==============================
# Thử với dấu phẩy làm separator
books = pd.read_csv(path + "/Books.csv", encoding="ISO-8859-1", sep=",", on_bad_lines="skip")
ratings = pd.read_csv(path + "/Ratings.csv", encoding="ISO-8859-1", sep=",", on_bad_lines="skip")

print("Books columns:", books.columns.tolist())
print("Ratings columns:", ratings.columns.tolist())

# ==============================
# 3. Gộp bảng theo ISBN
# ==============================
if "ISBN" in books.columns and "ISBN" in ratings.columns:
    df = books.merge(ratings, on="ISBN", how="inner")
    print("✅ Merge thành công! Kích thước:", df.shape)
else:
    print("⚠️ Không tìm thấy cột ISBN trong một trong hai file!")
    df = None

    # 4. Hàm gợi ý
    # ==============================
    def get_recommendations(preferences, data):
        filtered_data = data.copy()
        for feature, value in preferences.items():
            if value:
                if feature == "Book-Rating":  
                    filtered_data = filtered_data[filtered_data[feature] >= value]
                elif feature == "Year-Of-Publication":  
                    filtered_data = filtered_data[filtered_data[feature] == value]
                else:
                    filtered_data = filtered_data[filtered_data[feature].str.contains(str(value), case=False, na=False)]
        return filtered_data

    # ==============================
    # 5. Test gợi ý sách
    # ==============================
    user_preferences = {
        "Book-Author": "J.K. Rowling",
        "Publisher": "Scholastic",
        "Year-Of-Publication": 2000,
        "Book-Rating": 4
    }

    recommendations = get_recommendations(user_preferences, df)
    print("Recommended Books:")
    print(recommendations.head(10))


Path to dataset files: C:\Users\Lenovo\.cache\kagglehub\datasets\arashnic\book-recommendation-dataset\versions\3


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10628\2668547652.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path + "/Books.csv", encoding="ISO-8859-1", sep=",", on_bad_lines="skip")


Books columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
Ratings columns: ['User-ID', 'ISBN', 'Book-Rating']
✅ Merge thành công! Kích thước: (1031136, 10)


In [31]:
df = df[df['Book-Rating'] > 0]
def get_recommendations(author=None, publisher=None, year=None, min_rating=None, top_n=10):
    result = df.copy()
    
    if author:
        result = result[result['Book-Author'].str.contains(author, case=False, na=False)]
    if publisher:
        result = result[result['Publisher'].str.contains(publisher, case=False, na=False)]
    if year:
        result = result[result['Year-Of-Publication'] == year]
    if min_rating:
        result = result[result['Book-Rating'] >= min_rating]
    
    return result[['Book-Title','Book-Author','Publisher','Year-Of-Publication','Book-Rating']].drop_duplicates().head(top_n)

# Ví dụ test:
print(get_recommendations(author="J.K. Rowling", min_rating=5, top_n=5))


                                               Book-Title   Book-Author  \
458490  Harry Potter and the Prisoner of Azkaban (Book...  J.K. ROWLING   
458491  Harry Potter and the Prisoner of Azkaban (Book...  J.K. ROWLING   
458492  Harry Potter and the Prisoner of Azkaban (Book...  J.K. ROWLING   
501731  Harry Potter and the Prisoner of Azkaban (Harr...  J.K. Rowling   
501736  Harry Potter and the Prisoner of Azkaban (Harr...  J.K. Rowling   

                    Publisher Year-Of-Publication  Book-Rating  
458490      Listening Library                2000            9  
458491      Listening Library                2000           10  
458492      Listening Library                2000            8  
501731  Scholastic Paperbacks                2004            9  
501736  Scholastic Paperbacks                2004            8  


#### 2. Hotel Reviews Dataset

- Link: https://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe

- Description: This dataset provides hotel reviews, ratings, and attributes, making it useful for recommending hotels based on user preferences.

- Using Important Columns in Recommender Steps: Hotel Name, Hotel Address, Average Rating, Hotel Facilities, Hotel Location.

- Steps for Building the Recommender:

*Data Preparation*: Load and inspect Hotel Name, Hotel Address, Average Rating, Hotel Facilities, and Hotel Location.

*Feature Selection*: Use Hotel Name as the main identifier, with the other columns for filtering.

*Filtering Logic*: Allow filtering by Hotel Location and Hotel Facilities, and use Average Rating to recommend only hotels above a certain rating.

*User Preferences*: Collect user preferences for each of the selected columns.

*Recommendation Output*: Display hotels matching the specified preferences.

#### 3. Fashion Products on E-Commerce Platforms Dataset

- Link: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset

- Description: This dataset includes fashion product images, descriptions, and attributes, which can be used to create a fashion item recommender.

- Using Important Columns in Recommender Steps: Product Name, Category, Subcategory, Price, Brand, Product Type.

- Steps for Building the Recommender:

*Data Preparation*: Load the dataset and inspect Product Name, Category, Subcategory, Price, Brand, and Product Type.

*Feature Selection*: Use Product Name as the main identifier, with other columns for user-defined filters.

*Filtering Logic*: Enable filters for Category, Subcategory, Brand, and Product Type, with Price allowing range-based filtering.

*User Preferences*: Allow users to specify their desired category, subcategory, price range, brand, and type.

*Recommendation Output*: Display products that match the specified preferences.